In [2]:
from bs4 import BeautifulSoup
# basic
import numpy as np
import pandas as pd
import pymongo
# visual
import matplotlib.pyplot as plt
%matplotlib inline
#time
import datetime 

#talib
import talib
from talib import abstract

#gui

import requests
import re
import io

In [3]:
def crawl_price(stock_id):
    now = int(datetime.datetime.now().timestamp())+86400
    url = "https://query1.finance.yahoo.com/v7/finance/download/" + stock_id + "?period1=0&period2=" + str(now) + "&interval=1d&events=history&crumb=hP2rOschxO0"

    response = requests.post(url)
    
    f = io.StringIO(response.text)
    df = pd.read_csv(f, index_col='Date', parse_dates=['Date'] )
    return df

In [4]:
def get_webservice(url):
    list_req = requests.get(url)
    soup = BeautifulSoup(list_req.content)
    r = soup.find('string').text
    return r

In [5]:
def putOrder(bsaction, stockid,  ordertype, volume, price):
        url = "http://61.220.30.176/WebOrder/GVETransacs.asmx/PutOrderXML3?GMRIDStr=BCSP&CompCode="+stockid+"&Price="+price+"&Volume="+volume+"&BSAction="+bsaction+"&OrderType="+ordertype+"&IsOddLot=0&Currency=TWD&OrderNote=ROD&OCType=0&CombineNo=&OrderParameter=0&Lang=TC&str_ip=127.0.0.1"
        client = get_webservice(url)
        
        """try:
            import xml.etree.cElementTree as ET
        except ImportError:
            import xml.etree.ElementTree as ET
        r = ET.fromstring(client)
        """
        r = client.split("\"")
        if r[1] == "Success":
            return "委託下單成功"
            #tk.messagebox.showinfo(title='回報', message='Success')
        elif r[1] == "Failure":
            return "委託下單失敗"+'原因:'+r[5]
            #tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:'+new[5])
        else:
            return "委託下單失敗"+'原因:格式輸入錯誤'
            #tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:格式輸入錯誤')

In [6]:
#寫其他兩個回測
def buy(k):
    if k<20:
        print("buy") #回傳訊息給使用者
    elif k>80:
        print("sell") 
    else:
        print("不做買賣")

In [7]:
def job():
    
    df = crawl_price("2330.TW")
    start_time = datetime.datetime.now()-datetime.timedelta(days=365)
    end_time = datetime.date.today()
    close = df.Close[start_time:end_time].dropna().astype(float)
    close2 = round(close, 2)
    openp = df.Open[start_time:end_time]
    high = df.High[start_time:end_time]
    low = df.Low[start_time:end_time]
    volumn = df.Volume[start_time:end_time]
    stock = {
        'close':close.dropna().astype(float),
        'open':openp.dropna().astype(float),
        'high':high.dropna().astype(float),
        'low':low.dropna().astype(float),
        'volume': volumn.dropna().astype(float),
    }
    def talib2df(talib_output):
        if type(talib_output) == list:
            ret = pd.DataFrame(talib_output).transpose()
        else:
            ret = pd.Series(talib_output)
        ret.index = stock['close'].index
        return ret;

    data = talib2df(talib.abstract.STOCH(stock,fastk_period=9,
                    slowk_period=3,
                    slowk_matype=0,
                    slowd_period=3,
                    slowd_matype=0))

    data.reset_index(inplace=True)

    K = data.iloc[0:,1]
    a=[]
    for i in range(len(K)):
        a.append(K[i])
    
    
    print(a[-1])
    
    return buy(a[-1])

    #return ans

In [8]:
import schedule
import time
schedule.every(10).seconds.do(job)

while True:
    schedule.run_pending()
    time.sleep(1)

ConnectionError: HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v7/finance/download/2317.TW?period1=0&period2=1573122216&interval=1d&events=history&crumb=hP2rOschxO0 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000215827DC358>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:

每30分鐘提醒一次